In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import seaborn as sns
import matplotlib as mpl
plt.rcParams["figure.figsize"] = (15,10)
mpl.rcParams['font.size'] = 16  # Set the font size for all elements


from copy import deepcopy


#For random forest for time series:
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

#For metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

#Having to import from subfolders of parent directories is a nightmare
import sys
import os
current = os.path.dirname(os.path.realpath("high_to_low.py"))
parent = os.path.dirname(current)
sys.path.append(parent+"\Functions")

#From Functions folder
from LoadSeries import load_series, moving_average
from Merge import merge
from Battery import Battery
from DPModel import DP, DP_carb, DP_both
from Logic import logic_rollout, action_rollout, pred_logic_rollout, print_price_summary, logic_series_print
from MPC import MPC, MPC_carb, MPC_both, MPCModel
from Predictions import RF
from SARIMAfx import SARIMA

C:\Users\alexa\AppData\Roaming\Python\Python39\site-packages\statsmodels\compat\pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


# High

In [2]:
merge("h16").loc["2022-07-02 00:00:00":"2022-07-08 23:00:00"]

,prod_h16,cons_h16,yield,SpotPriceDKK,CO2Emission
Time,,,,,
2022-07-02 00:00:00,0.0,6.4,-6.4,2125.239990,175.0
2022-07-02 01:00:00,0.0,5.5,-5.5,1896.739990,169.0
2022-07-02 02:00:00,0.0,4.8,-4.8,1678.880005,139.0
2022-07-02 03:00:00,0.0,3.9,-3.9,1716.069946,119.0
2022-07-02 04:00:00,0.0,3.9,-3.9,1650.160034,189.0
...,...,...,...,...,...
2022-07-08 19:00:00,8.7,9.9,-1.2,1598.599976,64.0
2022-07-08 20:00:00,4.6,8.7,-4.1,1608.050049,56.0
2022-07-08 21:00:00,1.7,7.6,-5.9,1682.000000,78.0


In [3]:
merge("h16").loc["2022-01-08 00:00:00":"2022-01-14 23:00:00"]

,prod_h16,cons_h16,yield,SpotPriceDKK,CO2Emission
Time,,,,,
2022-01-08 00:00:00,0.0,5.1,-5.1,1138.239990,219.0
2022-01-08 01:00:00,0.0,4.0,-4.0,1086.540039,223.0
2022-01-08 02:00:00,0.0,4.1,-4.1,1075.829956,239.0
2022-01-08 03:00:00,0.0,3.9,-3.9,1084.010010,208.0
2022-01-08 04:00:00,0.0,3.5,-3.5,1093.160034,188.0
...,...,...,...,...,...
2022-01-14 19:00:00,0.0,12.5,-12.5,1082.500000,190.0
2022-01-14 20:00:00,0.0,8.0,-8.0,1019.479980,203.0
2022-01-14 21:00:00,0.0,6.3,-6.3,1083.550049,227.0


# Low

In [2]:
merge("h16").loc["2022-07-02 00:00:00":"2022-07-08 23:00:00"]

,prod_h16,cons_h16,yield,SpotPriceDKK,CO2Emission
Time,,,,,
2022-07-02 00:00:00,0.0,6.4,-6.4,186.520004,175.0
2022-07-02 01:00:00,0.0,5.5,-5.5,179.800003,169.0
2022-07-02 02:00:00,0.0,4.8,-4.8,169.649994,139.0
2022-07-02 03:00:00,0.0,3.9,-3.9,160.690002,119.0
2022-07-02 04:00:00,0.0,3.9,-3.9,153.529999,189.0
...,...,...,...,...,...
2022-07-08 19:00:00,8.7,9.9,-1.2,298.000000,64.0
2022-07-08 20:00:00,4.6,8.7,-4.1,235.539993,56.0
2022-07-08 21:00:00,1.7,7.6,-5.9,232.479996,78.0


In [3]:
merge("h16").loc["2022-01-08 00:00:00":"2022-01-14 23:00:00"]

,prod_h16,cons_h16,yield,SpotPriceDKK,CO2Emission
Time,,,,,
2022-01-08 00:00:00,0.0,5.1,-5.1,133.960007,219.0
2022-01-08 01:00:00,0.0,4.0,-4.0,156.139999,223.0
2022-01-08 02:00:00,0.0,4.1,-4.1,58.090000,239.0
2022-01-08 03:00:00,0.0,3.9,-3.9,107.000000,208.0
2022-01-08 04:00:00,0.0,3.5,-3.5,138.589996,188.0
...,...,...,...,...,...
2022-01-14 19:00:00,0.0,12.5,-12.5,377.079987,190.0
2022-01-14 20:00:00,0.0,8.0,-8.0,370.290009,203.0
2022-01-14 21:00:00,0.0,6.3,-6.3,330.660004,227.0


# We're missing the first 240 hours of prices

In [6]:
merge("h16").loc["2020-12-22 00:00:00":"2021-01-01 00:00:00"]

,prod_h16,cons_h16,yield,SpotPriceDKK,CO2Emission
Time,,,,,
2020-12-22 00:00:00,0.0,6.3,-6.3,-100.000000,60.0
2020-12-22 01:00:00,0.0,4.8,-4.8,-100.000000,58.0
2020-12-22 02:00:00,0.0,4.8,-4.8,-100.000000,65.0
2020-12-22 03:00:00,0.0,4.3,-4.3,-100.000000,67.0
2020-12-22 04:00:00,0.0,4.0,-4.0,-100.000000,76.0
...,...,...,...,...,...
2020-12-31 20:00:00,0.0,13.0,-13.0,-100.000000,230.0
2020-12-31 21:00:00,0.0,12.3,-12.3,-100.000000,193.0
2020-12-31 22:00:00,0.0,6.6,-6.6,-100.000000,200.0
